# BART Finetuning

By: Federico Dominguez Molina

#### Libraries

In [1]:
# !pip install transformers datasets evaluate rouge_score
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

import os
import numpy as np
import pandas as pd
import evaluate

# Local imports
from text_parser import TextParser

C:\Users\fdmol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fdmol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fdmol\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
MODEL_NAME = "facebook/bart-large-cnn"
TEXT_FILES_PATH = (
    "C:/Users/fdmol/Desktop/MSCAPP/CAPP30255/NLP-Police-Complaints/data/text_files"
)
DATA_PATH = "C:/Users/fdmol/Desktop/MSCAPP/CAPP30255/NLP-Police-Complaints/data"


In [3]:
# Add local parameters
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

rouge = evaluate.load("rouge")

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)


### Create and format training data

In [4]:
text_parser = TextParser(TEXT_FILES_PATH, nlp_task="summarization")

Initializing parsers for summarization


In [5]:
training_data = pd.ExcelFile(DATA_PATH + "/manual_summaries.xlsx")
training_data = training_data.parse("manual_summaries")

complete_texts = []
processed_manual_summaries = []
complaint_files = []

# Get local
for index, row in training_data.iterrows():
    manual_summary = row["manual_summary"]
    complaint_file = row["complaint"]

    # Get complete, preprocessed text
    complete_text = text_parser.file_to_string(f"{TEXT_FILES_PATH}/{complaint_file}")

    # Preprocess manual summary
    processed_manual_summary = text_parser.process_given_text(manual_summary)

    complete_texts.append(complete_text)
    processed_manual_summaries.append(processed_manual_summary)
    complaint_files.append(complaint_file)


all_text_files = os.listdir(TEXT_FILES_PATH)

# Get indices of files in the training set
training_set_indices = []

for file in all_text_files:
    if file in complaint_files:
        training_set_indices.append(all_text_files.index(file))

training_set = pd.DataFrame(
    {
        "complaint": complaint_files,
        "complete_text": complete_texts,
        "manual_summary": processed_manual_summaries,
        "file_index": training_set_indices,
    }
)


In [6]:
training_set.to_excel(DATA_PATH + "/training_set.xlsx")

Functions to finetune BART using the `transformers` library.

In [7]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

#### Tokenize data for model 

In [8]:
tokenized_inputs = tokenizer(
    training_set["complete_text"].tolist(),
    padding=True,
    truncation=True,
    max_length=1024,
)
tokenized_labels = tokenizer(
    training_set["manual_summary"].tolist(),
    padding=True,
    truncation=True,
    max_length=128,
)


#### Custom Dataset

In [9]:
import torch
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels["input_ids"][idx])
        return item

    def __len__(self):
        return len(self.labels["input_ids"])


dataset = CustomDataset(tokenized_inputs, tokenized_labels)

In [10]:
# training_args = Seq2SeqTrainingArguments(
#     output_dir="my_awesome_billsum_model",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     weight_decay=0.01,
#     save_total_limit=3,
#     num_train_epochs=4,
#     predict_with_generate=True,
#     fp16=True,
#     push_to_hub=True,
# )

# odel = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

training_args = TrainingArguments(
    output_dir="./results",  # Output directory for model checkpoints
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=4,  # Batch size per device during training
    per_device_eval_batch_size=4,  # Batch size for evaluation
    warmup_steps=500,  # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # Strength of weight decay
    logging_dir="./logs",  # Directory for storing logs
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    # eval_dataset=test_dataset,  # If you have a test dataset
)

trainer.train()


100%|██████████| 21/21 [20:51<00:00, 59.62s/it]

{'train_runtime': 1251.9204, 'train_samples_per_second': 0.06, 'train_steps_per_second': 0.017, 'train_loss': 3.3641651698521207, 'epoch': 3.0}


TrainOutput(global_step=21, training_loss=3.3641651698521207, metrics={'train_runtime': 1251.9204, 'train_samples_per_second': 0.06, 'train_steps_per_second': 0.017, 'train_loss': 3.3641651698521207, 'epoch': 3.0})

In [12]:
# Save model
model.save_pretrained(DATA_PATH + "/finetuned_bart_model")
tokenizer.save_pretrained(DATA_PATH + "/finetuned_bart_model")
